In [1]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

import os, sys, time
from datetime import datetime

import random
import scipy.stats as st
import pandas as pd
import numpy as np
from math import ceil
from functools import reduce
from tqdm import tqdm

sys.path.insert(0, "../scripts")

from feature_extractor import FeatureExtractor

import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_PATH = '../data/raw/pinheiro'
data = pd.read_csv(os.path.join(DATA_PATH, 'data.csv'))

In [3]:
def data_aggregation(data, agg_level):
    agg_cols_by_level = {
        1: ['date'],
        2: ['date', 'store_nbr'],
        3: ['date', 'SECAO'],
        4: ['date', 'GRUPO'],
        5: ['date', 'SUBGRUPO'],
        6: ['date', 'store_nbr', 'SECAO'],
        7: ['date', 'store_nbr', 'GRUPO'],
        8: ['date', 'store_nbr', 'SUBGRUPO'],
        9: ['date', 'item_nbr'],
        10: ['date', 'item_nbr', 'store_nbr']
    }
    
    agg_sales =  data.groupby(
        agg_cols_by_level[agg_level]
    )['unit_sales'].mean().reset_index()
    
    merge_cols = agg_sales.columns.tolist()
    merge_cols.remove('unit_sales')
    
    agg_data = data.drop(columns='unit_sales')
    agg_data = agg_data.merge(agg_sales, on=merge_cols)
    agg_data = agg_data[data.columns]
    
    return agg_data

In [4]:
def categorical_features_encoding(grid_df):
    id_columns = ['date', 'item_nbr', 'store_nbr']
    feature_columns = [col for col in grid_df.columns if col not in id_columns]

    categorical_features = grid_df[id_columns]
    for feature in feature_columns:
        df_dummy = pd.get_dummies(grid_df[feature])
        df_dummy = df_dummy.rename(columns={
            col: feature + '_' + str(col) for col in df_dummy.columns
        })

        categorical_features = pd.concat([categorical_features, df_dummy], axis=1)
        
    return categorical_features

In [5]:
def root_mean_squared_scaled_error(y, y_pred, h, n):
    a = np.power(y[n:n+h] - y_pred[n:n+h], 2)
    a = (1/h)*np.sum(a)
    
    b = 0
    for t in range(1, n):
        b += np.power(y[t][0] - y[t-1][0], 2)
        
    b = (1/(n-1))*b
    
    return np.sqrt(a/b)

In [6]:
search_grid = {
    'boosting_type': ['gbdt', 'dart', 'goss', 'rf'],
    'num_leaves': [20, 40, 60, 80, 100],
    'max_depth': [-1, 5, 10, 20],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 400, 600, 800, 1000],
    'subsample_for_bin': [20000, 40000, 80000, 160000],
    'class_weight': [None, 'balanced'],
    'min_child_samples': [20, 40, 60, 80, 100],
    'reg_alpha': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
    'reg_lambda': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]
}

In [7]:
samples = []
for item_nbr in data['item_nbr'].unique():
    for store_nbr in data['store_nbr'].unique():
        samples.append((item_nbr, store_nbr))

In [14]:
TRAIN_RATIO = 0.9
AGG_LEVEL = 2

results = {}

agg_data = data_aggregation(data, AGG_LEVEL)
feature_extractor = FeatureExtractor(agg_data)
features_dict = feature_extractor.extract()

num_features = features_dict['numeric']
cat_features = categorical_features_encoding(features_dict['categorical'])
features = num_features.merge(cat_features, on=['date', 'item_nbr', 'store_nbr'])

rmsse_list = []
for sample in tqdm(
    samples,
    bar_format='{l_bar}{bar:30}{r_bar}{bar:-30b}'
):
    item_nbr, store_nbr = sample
    try:
        item_mask = features['item_nbr'] == item_nbr
        store_mask = features['store_nbr'] == store_nbr

        df = features[item_mask & store_nbr]
        df_target = df[['unit_sales']]
        date_list = df['date'].tolist()
        df_features = df.drop(columns=['date', 'item_nbr', 'store_nbr', 'unit_sales'])

        train_size = int(TRAIN_RATIO*len(df_features))

        X = df_features.values
        y = df_target.values

        X_train, y_train = X[:train_size], y[:train_size]
        X_test, y_test = X[train_size:], y[train_size:]

        reg = LGBMRegressor()

        random_grid_search = RandomizedSearchCV(
            estimator=reg,
            param_distributions=search_grid,
            n_iter=5,
            cv=3,
            verbose=2,
            random_state=42,
            n_jobs=-1)

        random_grid_search.fit(X_train, y_train)

        y_pred = random_grid_search.predict(X_test)
        y_pred = y_pred.reshape(-1, 1)

        df_test = df.iloc[train_size:]
        df_pred = df_test[['date', 'item_nbr', 'store_nbr']]
        df_pred['unit_sales_pred'] = y_pred
        df_pred.to_csv(f'results/predictions/{item_nbr}_{store_nbr}.csv', index=False)
    except Exception as e:
        print(e)

INFO:FeatureExtractor:Extracting date features
INFO:FeatureExtractor:Extracting price features
INFO:FeatureExtractor:Extracting lagged features
INFO:FeatureExtractor:Features extracted successfully
  0%|                              | 0/1936 [00:00<?, ?it/s]

Fitting 3 folds for each of 5 candidates, totalling 15 fits


  0%|                              | 1/1936 [00:04<2:12:22,  4.10s/it]

Fitting 3 folds for each of 5 candidates, totalling 15 fits


  0%|                              | 2/1936 [00:07<2:03:54,  3.84s/it]

...

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=0.
Fitting 3 folds for each of 5 candidates, totalling 15 fits


100%|█████████████████████████████▉| 1932/1936 [2:42:48<00:03,  1.31it/s]

Fitting 3 folds for each of 5 candidates, totalling 15 fits


100%|██████████████████████████████| 1936/1936 [2:42:49<00:00,  5.05s/it]

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=0.
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=0.
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=0.


In [15]:
PREDICTIONS_PATH = 'results/predictions'

In [19]:
df_pred = pd.DataFrame()
for file in os.listdir(PREDICTIONS_PATH):
    pred = pd.read_csv(os.path.join(PREDICTIONS_PATH, file))
    df_pred = pd.concat([df_pred, pred])

In [22]:
df_pred.to_csv('results/df_pred.csv', index=False)